# Recognition of abnormality from Breast Thermograms Using ConvXGB

## Table of Contents
- [1 - Packages](#1)
- [2 - Pre - Processing ( Anisotropic Diffusion Filtering)](#2)
- [3 - Load the Dataset](#3)
- [4 - Test and Train Data](#4)
- [5 - CNN Model](#5)
- [6 - Json File ( For Storing Weights)](#6)
- [7 - Loading CNN Model](#7)
- [8 - XGBoost](#8)
- [9 - Evaluation Metric](#9)

<a name='1'></a>
# 1 - Packages

In [1]:
import numpy as np
import os
import pickle
import keras
from scipy import stats
import tensorflow as tf
import xgboost as xgb
from datetime import datetime
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import optimizers
from keras import regularizers
from keras.callbacks import LearningRateScheduler
import cv2
import warnings

<a name='2'></a>
# 2- Pre - Processing ( Anisotropic Diffusion Filtering)

In [2]:
def anisodiff(img,niter=1,kappa=50,gamma=0.1,step=(1.,1.),option=1,ploton=False):
   
    # ...you could always diffuse each color channel independently if you
    # really want
    if img.ndim == 3:
        warnings.warn("Only grayscale images allowed, converting to 2D matrix")
        img = img.mean(2)

    # initialize output array
    img = img.astype('float32')
    imgout = img.copy()

    # initialize some internal variables
    deltaS = np.zeros_like(imgout)
    deltaE = deltaS.copy()
    NS = deltaS.copy()
    EW = deltaS.copy()
    gS = np.ones_like(imgout)
    gE = gS.copy()

    # create the plot figure, if requested
    if ploton:
        import pylab as pl
        from time import sleep

        fig = pl.figure(figsize=(20,5.5),num="Anisotropic diffusion")
        ax1,ax2 = fig.add_subplot(1,2,1),fig.add_subplot(1,2,2)

        ax1.imshow(img,interpolation='nearest')
        ih = ax2.imshow(imgout,interpolation='nearest',animated=True)
        ax1.set_title("Original image")
        ax2.set_title("Iteration 0")

        fig.canvas.draw()

    for ii in range(niter):

        # calculate the diffs
        deltaS[:-1,: ] = np.diff(imgout,axis=0)
        deltaE[: ,:-1] = np.diff(imgout,axis=1)

        # conduction gradients (only need to compute one per dim!)
        if option == 1:
            gS = np.exp(-(deltaS/kappa)**2.)/step[0]
            gE = np.exp(-(deltaE/kappa)**2.)/step[1]
        elif option == 2:
            gS = 1./(1.+(deltaS/kappa)**2.)/step[0]
            gE = 1./(1.+(deltaE/kappa)**2.)/step[1]

        # update matrices
        E = gE*deltaE
        S = gS*deltaS

        # subtract a copy that has been shifted 'North/West' by one
        # pixel. don't as questions. just do it. trust me.
        NS[:] = S
        EW[:] = E
        NS[1:,:] -= S[:-1,:]
        EW[:,1:] -= E[:,:-1]

        # update the image
        imgout += gamma*(NS+EW)

        if ploton:
            iterstring = "Iteration %i" %(ii+1)
            ih.set_data(imgout)
            ax2.set_title(iterstring)
            fig.canvas.draw()
            # sleep(0.01)

    return imgout

In [3]:
from keras.models import model_from_json

<a name='3'></a>
# 3 - Load the Dataset

The Data Set Consists of total 102 Breast Thermal Images From visual DMI - IR Data Set(https://visual.ic.uff.br/dmi/).
- 70 Images (Without Cancer)
- 32 Images (With Cancer)
- Data Set Zip Folder (https://drive.google.com/file/d/1ILEfL8uose4R7BQGQGZevCw7SCiel62o/view?usp=sharing)

In [4]:
path_test = "C:/Users/rsvmu/Downloads/Data"
CATEGORIES = ["Withcancer","Withoutcancer"]
IMG_SIZE_X = 480
IMG_SIZE_Y = 640

**Note:** Here, The createTrainingData Fucntion besides loading/creating the data for model the two main steps are taking place before giving it to the model:
 - First step - Is converting the RGB image **(480, 640, 3)** to Gray_scale **(480, 640)**
 - Second step - Using the above function **anisdiff** filter and apply that on Gray_scale Images.

In [5]:
training = []
def createTrainingData():
    for category in CATEGORIES:
        path = os.path.join(path_test, category)
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            img_array = cv2.imread(os.path.join(path,img),0) #Changing RGB to Gray_Scale 
            aniso = anisodiff(img_array) # Applying anistropic Diffusion Filter to Gray_scale Images
            new_array = cv2.resize(aniso, (IMG_SIZE_X, IMG_SIZE_Y))
            training.append([new_array, class_num])
createTrainingData()

In [6]:
X =[]
y =[]
for features, label in training:
    X.append(features)
    y.append(label)
X = np.array(X).reshape(-1, IMG_SIZE_X, IMG_SIZE_Y)

In [7]:
X = X.astype('float32')
X /= 255
from keras.utils import np_utils
Y = np_utils.to_categorical(y, 2)
print(Y[5])
print(Y.shape)

[1. 0.]
(102, 2)


<a name='4'></a>
# 4 - Test and Train Data
- Test Data - 20 Percent of Whole data 102 Images

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 4)

In [10]:
batch_size = 2
nb_classes = 2
nb_epochs = 5
img_rows, img_columns = 480, 640
img_channel = 3
nb_filters = 32
nb_pool = 2
nb_conv = 3

<a name='5'></a>
# 5 - CNN Model
- I have first Used two Convolutional Layers with no padding and for hidden layers took RELU Activation Function and for Output layer we took sigmoid Function.

In [11]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation=tf.nn.relu,
                           input_shape=(480, 640,1)),
    tf.keras.layers.MaxPooling2D((2, 2), strides=2),
    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D((2, 2), strides=2),
    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D((2, 2), strides=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(2,  activation=tf.nn.sigmoid)
])

In [12]:
y_train = np.array(y_train)
y_test = np.array(y_test)

In [13]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.fit(X_train, y_train, batch_size = batch_size, epochs = nb_epochs, verbose = 1, validation_data = (X_test, y_test))

Epoch 1/5
38/38 [==============================] - 21s 449ms/step - loss: 0.7315 - accuracy: 0.3684 - val_loss: 0.6931 - val_accuracy: 0.6154
Epoch 2/5
38/38 [==============================] - 16s 433ms/step - loss: 0.6914 - accuracy: 0.7105 - val_loss: 0.6572 - val_accuracy: 0.6154
Epoch 3/5
38/38 [==============================] - 17s 438ms/step - loss: 0.7010 - accuracy: 0.6184 - val_loss: 0.6931 - val_accuracy: 0.3846
Epoch 4/5
38/38 [==============================] - 17s 450ms/step - loss: 0.6931 - accuracy: 0.2895 - val_loss: 0.6931 - val_accuracy: 0.3846
Epoch 5/5
38/38 [==============================] - 17s 448ms/step - loss: 0.6931 - accuracy: 0.2895 - val_loss: 0.6931 - val_accuracy: 0.3846


<a name='6'></a>
# 6 - Json File ( For Storing Weights)
- After Completeing the CNN we are storing the weights into json file and read the file to give this weights to a XgBoost.

In [14]:
json_model = model.to_json()

In [15]:
with open("model.json", "w") as json_file:
    json_file.write(json_model)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [16]:
mean = np.mean(X_train,axis=(0,1,2))
std = np.std(X_train,axis=(0,1,2))
X_train = (X_train-mean)/(std+1e-7)
X_test = (X_test-mean)/(std+1e-7)

<a name='7'></a>
# 7 - Loading CNN Model

In [17]:
def load_cnn_model(X_test, y_test):
	# load json and create model
	json_file = open('model.json', 'r')
	loaded_model_json = json_file.read()
	json_file.close()
	loaded_model = model_from_json(loaded_model_json)
	# load weights into new model
	loaded_model.load_weights("model.h5")
	 
	# evaluate loaded model on test data
	opt_rms = optimizers.Adam(learning_rate=0.001,decay=1e-6)
	loaded_model.compile(
		loss='categorical_crossentropy',
		optimizer=opt_rms,
		metrics=['accuracy'])
	'''
	y_test_ = np_utils.to_categorical(y_test, 10)
	scores = loaded_model.evaluate(X_test, y_test_, batch_size=128, verbose=1)
	print('\nTest result: %.3f loss: %.3f\n' % (scores[1]*100,scores[0]))
	'''
	return loaded_model

In [18]:
cnn_model = load_cnn_model(X_test, y_test)
print("Loaded CNN model from disk")

Loaded CNN model from disk


In [19]:
def get_feature_layer(model, data):
	
	total_layers = len(model.layers)
	fl_index = total_layers - 2
	feature_layer_model = keras.Model(
		inputs=model.input,
		outputs=model.get_layer(index=fl_index).output)
	
	feature_layer_output = feature_layer_model.predict(data)
	
	return feature_layer_output

In [20]:
X_train_cnn =  get_feature_layer(cnn_model, X_train)
print("Features extracted of training data")
X_test_cnn = get_feature_layer(cnn_model, X_test)
print("Features extracted of test data\n")

Features extracted of training data
Features extracted of test data



<a name='8'></a>
# 8 - XGBoost

In [21]:
def xgb_model(X_train, y_train, X_test, y_test):

	dtrain = xgb.DMatrix(
		X_train,
		label=y_train
	)

	dtest = xgb.DMatrix(
		X_test,
		label=y_test
	)

	results = {}

	params = {
		'max_depth':12,
		'eta':0.05,
		'objective':'multi:softprob',
		'num_class':2,
		'eval_metric':'merror'
	}

	watchlist = [(dtrain, 'train'),(dtest, 'eval')]
	n_round = 200

	model = xgb.train(
		params,
		dtrain,
		n_round,
		watchlist,
		evals_result=results)

	pickle.dump(model, open("cnn_xgboost_final.pickle.dat", "wb"))

	return model

<a name='9'></a>
# 9 - Evalution Metric

- merror - Multiclass classification error rate. It is calculated as **#(wrong cases)/#(all cases)**

In [22]:
print("Build and save of CNN-XGBoost Model.")
model = xgb_model(X_train_cnn, y_train, X_test_cnn, y_test)

Build and save of CNN-XGBoost Model.
[0]	train-merror:0.05263	eval-merror:0.19231
[1]	train-merror:0.03947	eval-merror:0.15385
[2]	train-merror:0.03947	eval-merror:0.15385
[3]	train-merror:0.03947	eval-merror:0.15385
[4]	train-merror:0.03947	eval-merror:0.19231
[5]	train-merror:0.03947	eval-merror:0.19231
[6]	train-merror:0.03947	eval-merror:0.19231
[7]	train-merror:0.03947	eval-merror:0.19231
[8]	train-merror:0.03947	eval-merror:0.19231
[9]	train-merror:0.02632	eval-merror:0.19231
[10]	train-merror:0.02632	eval-merror:0.19231
[11]	train-merror:0.02632	eval-merror:0.15385
[12]	train-merror:0.02632	eval-merror:0.11538
[13]	train-merror:0.02632	eval-merror:0.15385
[14]	train-merror:0.02632	eval-merror:0.11538
[15]	train-merror:0.02632	eval-merror:0.15385
[16]	train-merror:0.02632	eval-merror:0.15385
[17]	train-merror:0.02632	eval-merror:0.15385
[18]	train-merror:0.02632	eval-merror:0.15385
[19]	train-merror:0.02632	eval-merror:0.15385
[20]	train-merror:0.02632	eval-merror:0.15385
[21]	tr

C:\Users\rsvmu\anaconda3\lib\site-packages\xgboost\core.py:525: FutureWarning: Pass `evals` as keyword args.  Passing these as positional arguments will be considered as error in future releases.
  warnings.warn(


[55]	train-merror:0.02632	eval-merror:0.15385
[56]	train-merror:0.02632	eval-merror:0.15385
[57]	train-merror:0.02632	eval-merror:0.15385
[58]	train-merror:0.01316	eval-merror:0.15385
[59]	train-merror:0.01316	eval-merror:0.15385
[60]	train-merror:0.01316	eval-merror:0.15385
[61]	train-merror:0.01316	eval-merror:0.15385
[62]	train-merror:0.01316	eval-merror:0.15385
[63]	train-merror:0.01316	eval-merror:0.15385
[64]	train-merror:0.01316	eval-merror:0.15385
[65]	train-merror:0.01316	eval-merror:0.15385
[66]	train-merror:0.01316	eval-merror:0.15385
[67]	train-merror:0.01316	eval-merror:0.15385
[68]	train-merror:0.01316	eval-merror:0.15385
[69]	train-merror:0.01316	eval-merror:0.15385
[70]	train-merror:0.01316	eval-merror:0.15385
[71]	train-merror:0.01316	eval-merror:0.15385
[72]	train-merror:0.01316	eval-merror:0.15385
[73]	train-merror:0.01316	eval-merror:0.15385
[74]	train-merror:0.01316	eval-merror:0.15385
[75]	train-merror:0.01316	eval-merror:0.15385
[76]	train-merror:0.01316	eval-mer

In [28]:
from sklearn.metrics import plot_confusion_matrix